In [ ]:
import os
import glob
import numpy as np
import cv2
from PIL import Image
import math

import PIL
from rembg import remove
from preprocessing.image_processing import remove_background_from_folder, remove_background_recursive

## Background Removal
- Remove the background of the image and save image as RGBA without a background

### Remove BG for single folder
- Folder has to contain images only

In [ ]:
target_folder = "example_data/Meshfleet_Eval/Results_Diagonal_1"
remove_background_recursive(target_folder)

### BG Removal for Parent Folder
- Parent contains mutliple subfolders (e.g. of different objects)

In [ ]:
parent_folder = "example_data/Meshfleet_Eval/generated"

for name in os.listdir(parent_folder):
    folder_path = os.path.join(parent_folder, name)
    # only run on directories
    if os.path.isdir(folder_path):
        print(f"Processing {folder_path!r}…")
        remove_background_recursive(folder_path)

## Front View Detection

### VLM Mode
- VLM mode can be used for front view detection when no reference (like Ground Truth) is available

In [ ]:
import torch
from preprocessing.image_processing import align_views, load_dino, load_dino_model, get_dino_transform, release_gpu

device = "cuda" if torch.cuda.is_available() else "cpu"

from transformers import InstructBlipProcessor, InstructBlipForConditionalGeneration
vlm_model = InstructBlipForConditionalGeneration.from_pretrained("Salesforce/instructblip-vicuna-7b")
vlm_processor = InstructBlipProcessor.from_pretrained("Salesforce/instructblip-vicuna-7b")
vlm_model.to(device)

""""
dino_model = load_dino(device)
dino_transform = get_dino_transform(224)
"""

In [ ]:
gt_folder = "example_data/Meshfleet_Eval/Ground_Truth"
generated_folder = "example_data/Meshfleet_Eval/generated/Results_Side_3"
prompt = "Is this a front view or a rear view?"
df = align_views(generated_folder, detection_mode="vlm", vlm_model = vlm_model, vlm_processor= vlm_processor, prompt = prompt, device=device)

### DINO Mode
- DINO Mode is more reliable than VLM Mode
- Can be used when there is a Ground Truth image for comparison available

In [ ]:
import torch
from preprocessing.image_processing import align_views, load_dino, load_dino_model, get_dino_transform, release_gpu

device = "cuda" if torch.cuda.is_available() else "cpu"

dino_model = load_dino(device)
dino_transform = get_dino_transform(224)

gt_folder = "example_data/Meshfleet_Eval/Ground_Truth"
generated_folder = "example_data/Meshfleet_Eval/Results_001_FULL"
df = align_views(generated_folder, detection_mode="dino", dino_model= dino_model, dino_transform= dino_transform, ground_truth_parent= gt_folder, device=device)


## Manual Image Reordering
- Can be used to manually restructure a subfolder --> Given index will be set as the new 000.png
- E.g. if 006.png is passed, it will be set as the new 000.png and the other images will be reordered --> Order of images stays the same but starting image is modified

### Reorder Parent Folder
- Reorder parent folder that contains multiple object folders

In [ ]:
from preprocessing.image_processing import reorder_images_in_parent_folder

target_folder = "example_data/Meshfleet_Eval/Results_001_FULL"
reorder_images_in_parent_folder(target_folder, 3)

### Reorder Single Folder

In [ ]:
from preprocessing.image_processing import reorder_images_in_folder

target_folder = "example_data/Meshfleet_Eval/Ground_Truth/ff2111f89c46cec6f8cb5b77257343ba2007574c4f47bbff93eaa0899f548486"
reorder_images_in_folder(target_folder,9)

## Viewpoint Alignment
- Due to the front view detection it can happen that the viewpoints are swapped which leads to the vehicle being "rotated" in the wrong direction
- With this function the user can swap the direction of the rotation by swapping the images

In [ ]:
from preprocessing.image_processing import restructure_images
restructure_images("example_data/Meshfleet_Eval/Results_001_FULL")

## Concatenate images into grid
- Just for visualization

In [ ]:
from preprocessing.image_processing import concatenate_images
output_file_name = "concatenated_gt.png"
concatenate_images(
    "example_data/Meshfleet_Eval/Ground_Truth", 
    output_image=output_file_name, 
    rows = 4, 
    target_identifier = "003.png",
    crop_width_percent=30,
    crop_height_percent=50)

## Equal Scaling
- Adapt both images to equal scaling 
- Width of original image is set and generated images width is set to the same value while keeping the aspect ration fixed
- Objects are maximized in the image

### Equal Scaling for a folder pair with no subfolders

In [ ]:
from preprocessing.image_processing import process_equal_scaling, process_equal_scaling_structure

process_equal_scaling_structure(
    ground_truth_parent="example_data/Meshfleet_Eval/Ground_Truth",
    generated_parent="example_data/Meshfleet_Eval/Results_002",
    canvas_size=(768,768),
    fill_ratio=0.9
)

### Equal Scaling for multiple folders
- Folder Structure is:
      ground_truth_parent/
          <object_id_1>/
              000.png, 001.png, ..., 011.png
          <object_id_2>/
              ...
      generated_parent/
          <object_id_1>/
              000.png, 001.png, ..., 011.png
          <object_id_2>/
              ...

In [ ]:
original_folder = "example_data/examples_08082025/Car_Sample_2/original"
generated_folder = "example_data/examples_08082025/Car_Sample_2/generated_side"

out_original_folder = "example_data/examples_08082025/Car_Sample_2/scaled/original"
out_generated_folder = "example_data/examples_08082025/Car_Sample_2/scaled/generated_side"

process_equal_scaling(original_folder, generated_folder,
                        out_original_folder, out_generated_folder,
                        canvas_size=(768,768), fill_ratio=0.95)
